In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
try:
    plt.style.use('rossidata')
except:
    sns.set_style('white')
    
import collections
import scipy as sp
import os
import glob
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold,train_test_split
import lightgbm as lgb
import swifter
import time

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# donwload data from here https://www.drivendata.org/competitions/63/genetic-engineering-attribution/data/
# place in data directory

test_values=pd.read_csv('../data/test_values.csv')
train_labels=pd.read_csv('../data/train_labels.csv')
train_values=pd.read_csv('../data/train_values.csv')

In [ ]:
lgbm_lables=train_labels[train_labels.columns[1:]].idxmax(1)

y_values=pd.factorize(lgbm_lables)[0]
x_values=train_values[train_values.columns[2:]]

In [ ]:
X_dataset, X_holdout, y_dataset, y_holdout = train_test_split(x_values, y_values, test_size=0.1, random_state=88)


In [ ]:
lgb_train = lgb.Dataset(X_dataset, y_dataset)


In [ ]:
results=[]

lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': len(set(y_dataset)),
    'metric': ['multi_error'],
    "learning_rate": 0.05,
     "num_leaves": 60,
     "max_depth": 9,
     "feature_fraction": 0.45,
     "bagging_fraction": 0.3,
     "reg_alpha": 0.15,
     "reg_lambda": 0.15,
#      "min_split_gain": 0,
      "min_child_weight": 0
                }


modelstart= time.time()
# Find Optimal Parameters / Boosting Rounds
lgb_cv = lgb.cv(
    params = lgbm_params,
    train_set = lgb_train,
    num_boost_round=2000,
    stratified=True,
    nfold = 5,
    verbose_eval=50,
    seed = 23,
    early_stopping_rounds=75)

loss = lgbm_params["metric"][0]
optimal_rounds = np.argmin(lgb_cv[str(loss) + '-mean'])
best_cv_score = min(lgb_cv[str(loss) + '-mean'])

print("\nOptimal Round: {}\nOptimal Score: {} + {}".format(
    optimal_rounds,best_cv_score,lgb_cv[str(loss) + '-stdv'][optimal_rounds]))

results = results.append({"Rounds": optimal_rounds,
                          "Score": best_cv_score,
                          "STDV": lgb_cv[str(loss) + '-stdv'][optimal_rounds],
                          "LB": None,
                          "Parameters": lgbm_params}, ignore_index=True)